# Modelo NLP: Entrenamiento con Hyperparameter Tuner

In [1]:
import datetime

In [3]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_4/part_05'


In [ ]:
!pip install -q sagemaker-experiments

In [4]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError

___

In [5]:
image=sagemaker.image_uris.retrieve(framework='blazingtext', 
                                    region=region, 
                                    version='1')
print(image)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


685385470294.dkr.ecr.eu-west-1.amazonaws.com/blazingtext:1


-------

In [6]:
train_channel = prefix + '/train'
s3_train_data = f's3://{bucket}/{train_channel}'
validation_channel = prefix + '/validation'
s3_validation_data = f's3://{bucket}/{validation_channel}'
print(s3_train_data)
print(s3_validation_data)

data_channels = {'train': s3_train_data, 
                 'validation': s3_validation_data}

s3://sagemaker-eu-west-1-467432373215/module_4/part_05/train
s3://sagemaker-eu-west-1-467432373215/module_4/part_05/validation


#### Experimento variando el learning rate

- Tenemos los siguentes hyperparametros que podemos optimizar: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/blazingtext_hyperparameters.html

In [7]:
now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
experiment_name = f'dbpedia-text-classification-lr-{now}'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a text classification model using dbpedia dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')

In [8]:
s3_output_location = f's3://{bucket}/{prefix}/output-lr'

for lr in [0.1, 0.01, 0.001]:
    now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    jobname = f'dbpedia-blazingtext-{now}'

    # Creating a new trial for the experiment
    exp_trial = Trial.create(
        experiment_name=experiment_name, 
        trial_name=jobname)

    experiment_config={
        'ExperimentName': experiment_name,
        'TrialName': exp_trial.trial_name,
        'TrialComponentDisplayName': 'Training'}
    
    estimator = sagemaker.estimator.Estimator(
        image,
        role,
        instance_count=1,
        instance_type='ml.c5.2xlarge',
        volume_size=30,
        max_run=3600,
        input_mode='File',
        enable_sagemaker_metrics=True,
        output_path=s3_output_location,
        hyperparameters={
            'mode': 'supervised',
            'epochs': 40,
            'min_count': 2,
            'learning_rate': lr,
            'vector_dim': 10,
            'early_stopping': True,
            'patience': 4,
            'min_epochs': 5,
            'word_ngrams': 2
        },
    )
    
    estimator.fit(
        inputs=data_channels,
        job_name=jobname,
        experiment_config=experiment_config,
        wait=False, # This allow to continue in the for loop
    )
    print(f'Submitted training job {jobname}')

INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2024-05-24-13-37-46


Submitted training job dbpedia-blazingtext-2024-05-24-13-37-46


INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2024-05-24-13-37-48


Submitted training job dbpedia-blazingtext-2024-05-24-13-37-48


INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2024-05-24-13-37-49


Submitted training job dbpedia-blazingtext-2024-05-24-13-37-49


#### Hyperparameter tuner search
- Amazon SageMaker HyperparameterTuner busca la mejor versión de un modelo al ejecutar muchos trabajos de entrenamiento en el conjunto de datos mediante el algoritmo y los rangos de hiperparámetros que especifique.
- Elige los valores de hiperparámetros que dan lugar a un modelo con el mejor rendimiento medido por una métrica.
- https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html
- Esta vez usaremos spot instances: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/model-managed-spot-training.html

In [18]:
s3_output_location = f's3://{bucket}/{prefix}/output-hyper-opt'

jobname = f'dbpedia-blazingtext-hyper-new'

estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    use_spot_instances=True, # They can stop our trainings
    max_run=60*10,
    max_wait=60*10,
    volume_size=30,
    input_mode='File',
    enable_sagemaker_metrics=True,
    output_path=s3_output_location,
    hyperparameters={
        'mode': 'supervised',
        'epochs': 40,
        'min_count': 2,
        #'vector_dim': 10,
        #'learning_rate': 0.05,
        'early_stopping': True,
        'patience': 4,
        'min_epochs': 5,
        #'word_ngrams': 2
    },
)


In [19]:
# https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange
hyperparameter_ranges = {
    "learning_rate": sagemaker.parameter.ContinuousParameter(min_value=0.001, max_value=0.1, scaling_type='Logarithmic'),
    "vector_dim": sagemaker.parameter.IntegerParameter(min_value=5, max_value=15),
    "word_ngrams": sagemaker.parameter.IntegerParameter(min_value=1, max_value=5),
}

In [20]:
# https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html
tuner = sagemaker.tuner.HyperparameterTuner(
    estimator,
    "validation:accuracy",
    hyperparameter_ranges,
    objective_type='Maximize',
    max_jobs=20,
    max_parallel_jobs=10,
    strategy="Random",
)

In [ ]:
tuner.fit(    
    inputs=data_channels,
    job_name=jobname,
)

INFO:sagemaker:Creating hyperparameter tuning job with name: dbpedia-blazingtext-hyper-new


...................

- Podemos ver los resultados con HyperparameterTuningJobAnalytics.
- También podemos verlo en la pantalla de experimentos.

In [13]:
df= sagemaker.HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.job_name
).dataframe()
df

AttributeError: 'NoneType' object has no attribute 'job_name'

In [ ]:
df.sort_values(by='FinalObjectiveValue')

In [ ]:
df.loc[
    :, ['FinalObjectiveValue', 'learning_rate']
].set_index('learning_rate').sort_index().plot()

In [ ]:
df.loc[
    :, ['FinalObjectiveValue', 'vector_dim']
].set_index('vector_dim').sort_index().plot()

In [ ]:
df.loc[
    :, ['FinalObjectiveValue', 'word_ngrams']
].set_index('word_ngrams').sort_index().plot()